Стратегия объединяет BBANDS, RSI и определенные свечные паттерны для обнаружения точек разворота и определения оптимальных позиций входа.
- RSI - потдверждение тренда
- ATR - управления торговлей, используется для подсчета параметров стоп-лосса и тейк-профита (класс BBRSIStrategy [backtest_strategy.ipynb](backtest_strategy.ipynb))
- get_best_bb_rsi_strategy, backest_bb_rsi_strategy, apply_bb_rsi_strategy, add_bb_rsi_signal, walk_forward_optimization ([useful_functions.ipynb](useful_functions.ipynb))


In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# period=-(datetime.now() - (datetime.now() - relativedelta(years = 8))).days
period=-(datetime.now() - datetime(2016, 1, 1)).days
time_interval='1d'
tickers = ['EURUSD=X', 'GBPUSD=X', 'USDCHF=X', 'USDJPY=X']
forex_tickers = ['EURUSD', 'GBPUSD', 'USDCHF', 'USDJPY']
dir = 'forex_data'


In [ ]:
%run forex_data_loader.ipynb

In [3]:
import pandas as pd
import warnings
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import import_ipynb

from tqdm import tqdm
tqdm.pandas()
warnings.simplefilter('ignore')

### Usefull functions

In [5]:
# %run file_loader.ipynb
# %run useful_functions.ipynb
# %run backtest_strategy.ipynb

from file_loader import loading_v3, get_data
from useful_functions import get_best_bb_rsi_strategy, walk_forward_optimization_by_index, apply_bb_rsi_ema_strategy, add_pointpos_column, walk_forward_optimization_by_date_range2

from backtest_strategy import StopLossStrategy2

# log.basicConfig(level=log.INFO)

### Walk Forward Optimization
### Calculate Bollinger Bands and RSI,ATR and Signal apply

#### WFO by index

In [ ]:
report_dict = {}
symbols =['EURUSD']
for name in symbols:
# for name in forex_tickers:
    df = get_data(dir, name)
    df.reset_index(inplace=True)

    df_length = len(df)
    print(f"Start {name} backtesting, df: {df_length}")
   
    try:
        signals_df = pd.DataFrame()
       
        train_size = 180
        test_size = 33
        index_range = walk_forward_optimization_by_index(len(df), train_size, test_size)
        print(f"{name} wfo iterations: {len(index_range)}")

        params = {
            'bb_period_list': [15, 14, 13], # 13-30
            'bb_width_threshold_list': [0.002, 0.0015, 0.001],
            'atr_period_list': [7, 6, 5], # Long time frames: 20-50, Short: 2-10
            'rsi_period_list': [10, 11, 9], # Short-term intraday: 9-11, Medium-term: 14, Longer-term position: 20-30
            'cash': 10_000,
            # 'maximize': 'Profit Factor',
            'maximize': 'Return [%]',
            # 'maximize': 'Equity Final [$]',
            # 'maximize': 'Win Rate [%]',
            'max_tries': 300, # ???
            # 'stats_item': 'Profit Factor',
            'stats_item': 'Win Rate [%]',
            # 'method': 'skopt',
            'bb_std': 1.5, #1.8, 2
            'back_candles': 7, #8,9
            'ema_period': 30, # 20, 50, based on test_set_size param - ???            
        }

        for iter in tqdm(index_range):
            start_train = iter['train_indexes'][0]
            end_train = iter['train_indexes'][1]
            start_test = iter['test_indexes'][0]
            end_test = iter['test_indexes'][1]
            
            train_set = df.iloc[start_train:end_train].copy()
            test_set = df.iloc[start_test:end_test].copy()

            print(f"{name} : train_set: {train_set.iloc[0].Date} - {train_set.iloc[-1].Date}, test_set: {test_set.iloc[0].Date} - {test_set.iloc[-1].Date}")

            best_params, best_performance = get_best_bb_rsi_strategy(train_set.copy(), StopLossStrategy2, params, apply_bb_rsi_ema_strategy)

            print(f"{name} : best_params: {best_params}, best_performance: {best_performance}")
            if best_performance > 0:
                combined_data = pd.concat([train_set, test_set]).reset_index(drop=True)
                combined_with_test_signal = apply_bb_rsi_ema_strategy(combined_data.copy(), best_params)
                test_with_signal = combined_with_test_signal.iloc[-test_size:].copy()
                signals_df = pd.concat([signals_df, combined_with_test_signal], ignore_index=True)

        if not signals_df.empty:
            report_dict[name] = signals_df
    except Exception as e:
        print(f"Error backtesting for {name}: {e}")

### WFO by date index

In [ ]:
report_dict = {}
symbols =['EURUSD']
for name in symbols:
# for name in forex_tickers:
    df = get_data(dir, name)
    df.set_index('Date', inplace=True)

    df_length = len(df)
    print(f"Start {name} backtesting, df: {df_length}")

    try:
        train_size_in_month = 9
        test_size_in_month = 1
        dates_range = walk_forward_optimization_by_date_range2(df.index.date[0], df.index.date[-1], train_size_in_month, test_size_in_month)
        print(f"{name} wfo iterations: {len(dates_range)}")

        params = {
            'bb_period_list': [15, 20, 30], # 13-30
            'bb_width_threshold_list': [0.002, 0.0015, 0.001],
            'atr_period_list': [13, 15, 14], # Long time frames: 20-50, Short: 2-10
            'rsi_period_list': [13, 15, 14], # Short-term intraday: 9-11, Medium-term: 14, Longer-term position: 20-30
            'cash': 10_000,
            # 'maximize': 'Profit Factor',
            'maximize': 'Return [%]',
            # 'maximize': 'Equity Final [$]',
            # 'maximize': 'Win Rate [%]',
            # 'maximize': None, # SQN
            'max_tries': 300,
            # 'stats_item': 'Profit Factor',
            'stats_item': 'Win Rate [%]',
            # 'method': 'skopt',
            'bb_std': 1.5, #1.8, 2
            'back_candles': 7, #8,9
            'ema_period': 30, # 20, 50, based on test_set_size param - ???            
        }

        signals_df = pd.DataFrame()
        for iter in tqdm(dates_range):
            train_from_date = datetime.combine(iter['train_dates'][0], datetime.min.time())
            train_end_date = datetime.combine(iter['train_dates'][1], datetime.min.time())
            test_from_date = datetime.combine(iter['test_dates'][0], datetime.min.time())
            test_end_date = datetime.combine(iter['test_dates'][1], datetime.min.time())
            
            train_set = df[(df.index >= train_from_date) & (df.index <= train_end_date)].copy()
            test_set = df[(df.index >= test_from_date) & (df.index <= test_end_date)].copy()

            train_set.reset_index(inplace=True)
            test_set.reset_index(inplace=True)
            
            print(f"{name} : train_set: {train_set.iloc[0].Date} - {train_set.iloc[-1].Date}, test_set: {test_set.iloc[0].Date} - {test_set.iloc[-1].Date}")

            best_params, best_performance = get_best_bb_rsi_strategy(train_set, StopLossStrategy2, params, apply_bb_rsi_ema_strategy)

            print(f"{name} : best_params: {best_params}, best_performance: {best_performance}")
            if best_performance > 0:
                combined_data = pd.concat([train_set, test_set]).reset_index(drop=True)
                combined_with_test_signal = apply_bb_rsi_ema_strategy(combined_data.copy(), best_params).copy()
                combined_with_test_signal.set_index('Date', inplace=True)

                test_set_with_signal = combined_with_test_signal[(combined_with_test_signal.index >= test_from_date) &
                                                             (combined_with_test_signal.index <= test_end_date)
                                                            ].copy()
                signals_df = pd.concat([signals_df, test_set_with_signal], ignore_index=True)

        if not signals_df.empty:
            report_dict[name] = signals_df
    except Exception as e:
        print(f"Error backtesting for {name}: {e}")


Start EURUSD backtesting, df: 2338
EURUSD wfo iterations: 66


  0%|          | 0/99 [00:00<?, ?it/s]

EURUSD : train_set: 2016-01-01 00:00:00 - 2016-09-30 00:00:00, test_set: 2016-10-03 00:00:00 - 2016-10-31 00:00:00


100%|██████████| 196/196 [00:01<00:00, 112.16it/s]


Backtest.optimize:   0%|          | 0/9 [00:00<?, ?it/s]

...

Backtest.optimize:   0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 197/197 [00:04<00:00, 46.72it/s]


Backtest.optimize:   0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 197/197 [00:04<00:00, 44.69it/s]


Backtest.optimize:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
bt_df = forex_dict['BTC-USD'].copy()
# print(f"{bt_df.tail(10)}")

# # df.index.names = ['Datetime']
# # bt_df["Date"] = pd.to_datetime(bt_df["Datetime"])
# bt_df.set_index('Date', inplace=True)
bt_df = bt_df.sort_index()

bt = Backtest(bt_df, BBRSIStrategy, cash=10_0000, commission=0.002, exclusive_orders=True, margin=0.1)
stats = bt.run()
stats
# print(stats[:27])
# bt.plot(plot_equity=True,plot_drawdown=True,relative_equity=False,)


KeyError: "None of ['Date'] are in the columns"

### Checks the count of entry and exit signals for each ticker

In [55]:
sum([forex_dict[key]['Signal'].value_counts() for key in forex_dict], start=0)

Signal
0    31
Name: count, dtype: int64

In [56]:
# Checks the count of signals for each ticker
for df in forex_dict.values():
    print(len(df[df.Signal != 0]))

0


In [ ]:
# Add point of position
for df in forex_dict.values():
    df = add_pointpos_column(df, "Signal", 1e-4)

# forex_dict['EURUSD'].describe()

In [ ]:
st=2600
for name, df in forex_dict.items():
    dfpl = df[st:st+350]

    # Create a plot with 2 rows
    fig = make_subplots(rows=2, cols=1, column_titles=[name])

    # Add candlestick plot on the first row
    fig.add_trace(go.Candlestick(x=dfpl.index,
                                open=dfpl['Open'],
                                high=dfpl['High'],
                                low=dfpl['Low'],
                                close=dfpl['Close']),
                row=1, col=1)

    # Add Bollinger Bands, EMA lines on the same subplot
    fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['bb_low'],
                            line=dict(color='green', width=1),
                            name="BB_Low"),
                row=1, col=1)
    
    fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['bb_mid'],
                            line=dict(color='yellow', width=1),
                            name="BB_MID"),
                row=1, col=1)

    fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['bb_up'],
                            line=dict(color='green', width=1),
                            name="BB_UPPER"),
                row=1, col=1)

    # Add markers for trade entry points on the same subplot
    fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                            marker=dict(size=8, color="MediumPurple"),
                            name="entry"),
                row=1, col=1)

    # Add markers for trade entry points on the same subplot
    fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['rsi'], 
                            line=dict(color='brown', width=2),
                            name="RSI"),
                row=2, col=1)

    fig.update_layout(width=800, height=600, sliders=[])
    fig.show()


In [ ]:
%run backtest_strategy.ipynb

In [46]:
results = dict()

for name, df in forex_dict.items():
    bt = Backtest(df, BBRSIStrategy, cash=10000, margin=1/10, commission=0.002)
    stats, heatmap = bt.optimize(slcoef=[i/10 for i in range(10, 16)],
                    TPcoef=[i/10 for i in range(10, 21)],
                    maximize='Return [%]', max_tries=300,
                    random_state=0,
                    return_heatmap=True)
    results[name] = [stats, heatmap]


### Stats


In [ ]:
for name, result_list in results.items():
    print("=== Stats of {} ===".format(name))
    print(result_list[0])


### Heatmap

In [ ]:
import seaborn as sns

for name, result_list in results.items():
    print("=== heatmap of {} ===".format(name))
    heatmap_df = result_list[1].unstack()
    plt.figure(figsize=(10, 8))
    sns.heatmap(heatmap_df, annot=True, cmap='viridis', fmt='.0f')
    plt.show()